In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf 

from plotly.subplots import make_subplots

In [2]:

account_currency = 'USD'

In [3]:
myassets = pd.read_excel('personal_portfolio.xlsx')

Create the dataframe to follow the savings by time

In [4]:
savings_in_usd = pd.DataFrame()
asset_change = yf.download(f'TRYUSD=X',period='1d', start='2025-01-01')
savings_in_usd['Date'] = asset_change.reset_index()['Date']
savings_in_usd['Close Total'] = np.zeros(len(asset_change['Close']))
savings_in_usd = savings_in_usd.set_index("Date")

[*********************100%%**********************]  1 of 1 completed


Update the savings dataframe with the data acquired from yahoo-finance

In [5]:
for asset in myassets.itertuples():
    if asset.asset_type == 'fiat':
        asset_change = yf.download(f'{asset.asset}USD=X',period='1d', start='2025-01-01')
    elif asset.asset_type == 'stock':
        asset_change = yf.download(f'{asset.asset}',period='1d', start='2025-01-01')
    else: 
        display('asset type is unknown!')
    indices = asset_change.reset_index()['Date']   
    savings_in_usd.loc[indices, f'Close {asset.asset}'] = asset_change['Close'] * asset.amount
    
savings_in_usd = savings_in_usd.ffill()

for asset in myassets.itertuples(): 
    savings_in_usd['Close Total'] += savings_in_usd[f'Close {asset.asset}'] 

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Create a dataframe to image final savings' distribution in a pie chart

In [6]:
pie_chart_df = pd.DataFrame({'Close' : savings_in_usd.iloc[-1, 1:].array},
                            index = myassets.asset
                            )

Plot!

In [7]:
fig = make_subplots(rows = 1, cols = 2, 
                    specs=[[{'type':'xy'}, {'type':'domain'}]], 
                    column_widths=[0.7, 0.2],
                    subplot_titles = [f'Total savings: {np.sum(pie_chart_df['Close']) :.2f}$', f'Savings Distribution']
                    )

for asset in myassets.asset:
    line_chart = go.Scatter(x = savings_in_usd.reset_index()['Date'], y = savings_in_usd[f'Close {asset}'], mode = 'lines+markers', name = asset)
    fig.add_trace(line_chart, 1, 1)

asset = 'Total'
line_chart = go.Scatter(x = savings_in_usd.reset_index()['Date'], y = savings_in_usd[f'Close {asset}'], mode = 'lines+markers', name = asset)
fig.add_trace(line_chart, 1, 1)

pie_chart = go.Pie(labels = pie_chart_df.reset_index()['asset'], 
                   values = pie_chart_df['Close'], 
                   
                   textinfo = 'percent+label', 
                   hole = .3, 
                   marker = {'colors': px.colors.sequential.RdBu},
                   showlegend=False)
fig.add_trace(pie_chart, 1, 2)


fig.update_layout(height=500, width=1500)
# fig.update_traces(textposition='inside', textinfo='percent+label')
# fig.update_layout(title = f'Total savings: {np.sum(pie_chart_df['Close']) :.2f}$')
fig.show()